In [1]:
import numpy as np
import pandas as pd
import pickle
from gensim.models import word2vec
import MeCab
import sys
from collections import Counter
import re
import urllib.request, urllib.error

In [2]:
tagger = MeCab.Tagger("-Owakati")
tagger.parse("") # パーサーにデータを渡す前にこれを挟むことで、UnicodeDecodeErrorを避けることが出来る。
                 #具体的な理由は分かっていないが、おそらく一度tagger.parse('')を挟むことで、プログラム内で使用されている標準の文字エンコードで初期化されるのではないかと思う。

'\n'

In [3]:
lunch_file = ["pretest_sushitsuu_lunch.csv",
              "pretest_sushiichi_lunch.csv",
              "pretest_sawada_lunch.csv",
              "pretest_mitani_lunch.csv",
              "pretest_matsukan_lunch.csv",
              "pretest_kiyoda_lunch.csv",
              "pretest_imamura_lunch.csv",
              "pretest_hatsune_lunch.csv"
              ]

In [4]:
dinner_file = ["pretest_sushitsuu_dinner.csv",
              "pretest_sushiichi_dinner.csv",
              "pretest_sawada_dinner.csv",
              "pretest_mitani_dinner.csv",
              "pretest_matsukan_dinner.csv",
              "pretest_kiyoda_dinner.csv",
              "pretest_imamura_dinner.csv",
              "pretest_hatsune_dinner.csv",
              "pretest_aozora_dinner.csv"
              ]

In [5]:
store_name = ["sushitsuu",
             "sushiichi",
             "sawada",
             "mitani",
             "matsukan",
             "kiyoda",
             "imamura",
             "hatsune",
             "aozora",
             ]

In [6]:
url = 'http://svn.sourceforge.jp/svnroot/slothlib/CSharp/Version1/SlothLib/NLP/Filter/StopWord/word/Japanese.txt'
slothlib_file = urllib.request.urlopen(url=url)
stop_words = [line.decode("utf-8").strip() for line in slothlib_file]
stop_words = [sw for sw in stop_words if not sw==u'']
my_stop_words = ["℃/", "℃", "kg", "訪問", "寿司", "すし", "なく", "美味しい", "美味", "～」", "○－", "○○/", ")○/"]
stop_words += my_stop_words 

In [7]:
def normalize_number(text):
    # 連続した数字を0で置換
    replaced_text = re.sub(r'\d+', '0', text)
    return replaced_text

In [8]:
def tokenize_ja(text, lower):
    text = replaced_text = re.sub(r"\d+", "0", text)
    node = tagger.parseToNode(str(text))
    while node:
        surface = node.surface.lower()
        if lower and node.feature.split(',')[0] in ["名詞","形容詞"] and surface not in stop_words:
            #分かち書きで取得する品詞を指定
            yield surface
            #lowerで小文字に変換 正規化
        node = node.next

In [9]:
def tokenize(content, token_min_len, token_max_len, lower):
    return [
        str(token) for token in tokenize_ja(content, lower)
        if token_min_len <= len(token) <= token_max_len and not token.startswith('_')
    ]

In [18]:
def kyouki_lunch(path):
    df = pd.read_csv(path)
    wakati_sushi_text = []
    word_list = []
    many_words = []
    similar_words = []
    for i in df["lunch_review"]:
        txt = tokenize(i, 2, 10000, True)
        wakati_sushi_text.append(txt)
        for t in txt:
            word_list.append(t)
    #word2vec_sushi_model = word2vec.Word2Vec(word_list,sg=1,size=100, window=5,min_count=5,iter=100,workers=3)
    print("昼間" in word_list)
    word2vec_sushi_model = word2vec.Word2Vec(wakati_sushi_text,sg=1,size=100, window=5,min_count=4,iter=100,workers=3)
    counter = Counter(word_list)
    for word, count in counter.most_common(10):
        #word2vec_sushi_model = word2vec.Word2Vec(wakati_sushi_text,sg=1,size=100, window=5,min_count=5,iter=100,workers=3)
        many_words.append(word + " " + str(count))
        #print(word)
        #print(count)
        similar = word2vec_sushi_model.most_similar(word)
        #similar_words.append(similar[0] + " " + str(similar[1]))
        similar_words.append(similar)
    return many_words, similar_words

In [26]:
def kyouki_dinner(path):
    df = pd.read_csv(path)
    wakati_sushi_text = []
    word_list = []
    many_words = []
    similar_words = []
    for i in df["dinner_review"]:
        txt = tokenize(i, 2, 10000, True)
        wakati_sushi_text.append(txt)
        word_list.append(i)
        for t in txt:
            word_list.append(t)
    counter = Counter(word_list)
    word2vec_sushi_model = word2vec.Word2Vec(wakati_sushi_text,sg=1,size=100, window=5,min_count=3,iter=100,workers=3)
    for word, count in counter.most_common(10):
        many_words.append(word + " " + str(count))
        similar = word2vec_sushi_model.most_similar(word)
        #similar_words.append(similar[0] + " " + str(similar[1]))
        similar_words.append(similar)
    return many_words, similar_words

In [24]:
many_words_lunch = []
similar_words_lunch = []
for path in lunch_file:
    many_word, similar_word = kyouki_lunch(path)
    many_words_lunch.append(many_word)
    similar_words_lunch.append(similar_word)
    print("=====================")

True


/Users/yuki/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:21: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).


True
True
True
True
False
True
False


In [21]:
similar_words_lunch[0]

[[('展開', 0.45621126890182495),
  ('休日', 0.4144855737686157),
  ('お昼', 0.4027041792869568),
  ('邪魔', 0.390051007270813),
  ('定番', 0.3877781927585602),
  ('売り切れ', 0.3822166621685028),
  ('オープン', 0.3582955598831177),
  ('美術館', 0.3561379015445709),
  ('海老', 0.3554736077785492),
  ('アラ', 0.3459850251674652)],
 [('日間', 0.49058592319488525),
  ('別物', 0.4175192713737488),
  ('熱心', 0.40511661767959595),
  ('一言', 0.40416574478149414),
  ('成分', 0.382813960313797),
  ('顕微鏡', 0.37520086765289307),
  ('ムース', 0.3652326464653015),
  ('ピー', 0.363463819026947),
  ('口中', 0.3618115782737732),
  ('包丁', 0.36160415410995483)],
 [('休日', 0.4507294297218323),
  ('定食', 0.42913326621055603),
  ('投稿', 0.420721173286438),
  ('予約', 0.40972042083740234),
  ('ログ', 0.39995768666267395),
  ('時半', 0.3943234086036682),
  ('開店', 0.3887757360935211),
  ('美術館', 0.38621029257774353),
  ('久々', 0.3846472501754761),
  ('会社', 0.3812657296657562)],
 [('シャリ', 0.4445815682411194),
  ('満腹', 0.41783779859542847),
  ('大き', 0.4134100973

In [22]:
many_words_lunch

[['握り 196',
  '熟成 175',
  'ランチ 139',
  'ネタ 131',
  'シャリ 125',
  '大将 111',
  '予約 98',
  '海老 81',
  'トロ 76',
  'カウンター 75'],
 ['銀座 52',
  '握り 52',
  'ランチ 47',
  'コース 46',
  'カウンター 30',
  'シャリ 30',
  '穴子 26',
  '料理 25',
  '良い 25',
  'いい 24'],
 ['トロ 168',
  '握り 143',
  'シャリ 116',
  'さわ 104',
  'ネタ 89',
  '穴子 70',
  '澤田 68',
  'ない 66',
  '雲丹 65',
  '軍艦 63'],
 ['三谷 182',
  '予約 151',
  '料理 113',
  '握り 103',
  'ワイン 69',
  '雲丹 56',
  '日本 53',
  'トロ 50',
  '穴子 47',
  '巻き 47'],
 ['握り 12',
  'ネタ 10',
  'まつ 7',
  '予約 7',
  'トロ 7',
  '麻布 7',
  '雰囲気 6',
  '穴子 5',
  '店内 5',
  '昼間 4'],
 ['マグロ 39',
  'トロ 32',
  '握り 25',
  'きよ 23',
  '赤身 21',
  '旨味 17',
  '巻き 14',
  '最高 13',
  'ない 12',
  'まぐろ 11'],
 ['握り 90',
  'シャリ 59',
  'ネタ 53',
  '穴子 50',
  '良い 49',
  'トロ 49',
  '車海老 40',
  'ない 39',
  '予約 35',
  'いい 35'],
 ['シャリ 91',
  'トロ 84',
  '握り 81',
  'ネタ 79',
  '漬け 65',
  '大将 55',
  '旨み 54',
  '初音 47',
  '赤身 42',
  '香り 42']]

In [27]:
many_words_dinner = []
similar_words_dinner = []
for path in dinner_file:
    many_word, similar_word = kyouki_dinner(path)
    many_words_dinner.append(many_word)
    similar_words_dinner.append(similar_word)
    print("=====================")

/Users/yuki/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:17: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).


In [17]:
many_words_dinner

[['握り', '大将', '熟成', 'ネタ', 'トロ', '包丁', 'シャリ', '料理', '予約', 'カウンター'],
 ['銀座', '握り', 'カウンター', 'ワイン', '予約', 'コース', 'いい', '良い', 'ネタ', '雰囲気'],
 ['トロ', '握り', 'さわ', 'シャリ', '銀座', 'ネタ', '穴子', '大将', 'ない', '雲丹'],
 ['三谷', '予約', '料理', '握り', 'ワイン', '日本', 'マリアージュ', '雲丹', 'ない', '大将'],
 ['カウンター', '握り', '予約', 'いい', '値段', '麻布', '良い', 'ない', 'ネタ', '好き'],
 ['マグロ', '握り', 'きよ', 'まぐろ', 'トロ', '最高', '銀座', '旨い', '穴子', '大将'],
 ['握り', 'シャリ', 'トロ', '穴子', '良い', 'ネタ', '白金', '車海老', '大将', 'カウンター'],
 ['トロ', '握り', 'シャリ', '大将', 'ネタ', '初音', '漬け', '赤身', '予約', 'マグロ'],
 ['握り', 'シャリ', '穴子', 'トロ', '銀座', '青空', '大将', 'ネタ', 'すき', 'やばし']]

In [28]:
similar_words_dinner

[[[('メニュー', 0.3624200224876404),
   ('墨烏賊', 0.35913535952568054),
   ('タコマンマ', 0.3588726818561554),
   ('毛蟹', 0.3584284782409668),
   ('紫蘇', 0.3542347550392151),
   ('あん', 0.3538050651550293),
   ('計算', 0.3486643135547638),
   ('主役', 0.34401556849479675),
   ('サバ', 0.3426024317741394),
   ('交互', 0.34078338742256165)],
  [('努力', 0.40879690647125244),
   ('台湾', 0.382610023021698),
   ('予約', 0.3788611888885498),
   ('っ子', 0.37241649627685547),
   ('商売', 0.37005043029785156),
   ('独立', 0.3666098713874817),
   ('江戸', 0.36267536878585815),
   ('欲しい', 0.3595333993434906),
   ('ng', 0.3583586812019348),
   ('信念', 0.35209786891937256)],
  [('キロ', 0.44850271940231323),
   ('圧巻', 0.4381587505340576),
   ('新鮮', 0.4190056025981903),
   ('凝縮', 0.4141848683357239),
   ('極限', 0.40178659558296204),
   ('ネットリ', 0.39928972721099854),
   ('プリプリ', 0.39603155851364136),
   ('曰く', 0.39589011669158936),
   ('カット', 0.39512205123901367),
   ('隠し', 0.38643884658813477)],
  [('解説', 0.4562045931816101),
   ('包丁', 

In [19]:

#df = pd.read_csv('pretest_matsukan_lunch.csv')
##df_sushi = df.groupby(['store_name','score'])['review'].apply(list).apply(' '.join).reset_index().sort_values('score', ascending=False)
##コーパス作成
#wakati_sushi_text = []
#word_list = []
#for i in df["lunch_review"]:
    #txt = tokenize(i, 2, 10000, True)
    #wakati_sushi_text.append(txt)
## モデル作成
#word2vec_sushi_model = word2vec.Word2Vec(wakati_sushi_text,sg=1,size=100, window=5,min_count=5,iter=100,workers=3)
##sg（0: CBOW, 1: skip-gram）,size（ベクトルの次元数）,window（学習に使う前後の単語数）,min_count（n回未満登場する単語を破棄）,iter（トレーニング反復回数）
##workers:複数のスレッドで処理

In [40]:
def makedf_lunch(i):
    df = pd.DataFrame({many_words_lunch[i][0]:similar_words_lunch[i][0]})
    for t in range(1, len(similar_words_lunch[0])):
        df2 = pd.DataFrame({many_words_lunch[i][t]:similar_words_lunch[i][t]})
        df = pd.concat([df, df2], axis=1)
    return df.T

In [41]:
lunch = []
for i in range(len(store_name) - 1): 
    lunch.append(makedf_lunch(i))

In [45]:
dinner[0]

,0,1,2,3,4,5,6,7,8,9
握り 228,"(メニュー, 0.3624200224876404)","(墨烏賊, 0.35913535952568054)","(タコマンマ, 0.3588726818561554)","(毛蟹, 0.3584284782409668)","(紫蘇, 0.3542347550392151)","(あん, 0.3538050651550293)","(計算, 0.3486643135547638)","(主役, 0.34401556849479675)","(サバ, 0.3426024317741394)","(交互, 0.34078338742256165)"
大将 176,"(努力, 0.40879690647125244)","(台湾, 0.382610023021698)","(予約, 0.3788611888885498)","(っ子, 0.37241649627685547)","(商売, 0.37005043029785156)","(独立, 0.3666098713874817)","(江戸, 0.36267536878585815)","(欲しい, 0.3595333993434906)","(ng, 0.3583586812019348)","(信念, 0.35209786891937256)"
熟成 160,"(キロ, 0.44850271940231323)","(圧巻, 0.4381587505340576)","(新鮮, 0.4190056025981903)","(凝縮, 0.4141848683357239)","(極限, 0.40178659558296204)","(ネットリ, 0.39928972721099854)","(プリプリ, 0.39603155851364136)","(曰く, 0.39589011669158936)","(カット, 0.39512205123901367)","(隠し, 0.38643884658813477)"
ネタ 100,"(解説, 0.4562045931816101)","(包丁, 0.4347403645515442)","(感服, 0.4197380542755127)","(飾り, 0.41874465346336365)","(小さめ, 0.41203635931015015)","(使い, 0.4093402028083801)","(シャリ, 0.4086216688156128)","(醍醐味, 0.3972688913345337)","(すぎ, 0.3773961365222931)","(美しく, 0.36794501543045044)"
トロ 84,"(口当たり, 0.44059091806411743)","(唐墨, 0.43117615580558777)","(大間, 0.41539323329925537)","(ジュレ, 0.4145203232765198)","(包丁, 0.39789387583732605)","(あん, 0.3738864064216614)","(飾り, 0.3586828410625458)","(カンパチ, 0.35296496748924255)","(タコマンマ, 0.35168036818504333)","(すじ, 0.349490225315094)"
包丁 83,"(隠し, 0.5937645435333252)","(飾り, 0.5014629364013672)","(舌触り, 0.4423221945762634)","(ネタ, 0.4347403049468994)","(ピンセット, 0.42430299520492554)","(塩分, 0.40867018699645996)","(キロ, 0.4035183787345886)","(トロ, 0.39789384603500366)","(日間, 0.39393407106399536)","(大間, 0.38993310928344727)"
シャリ 77,"(現代, 0.40937966108322144)","(解説, 0.40923839807510376)","(ネタ, 0.408621609210968)","(小さめ, 0.40724241733551025)","(サバ, 0.4007398188114166)","(ズワイ, 0.39923572540283203)","(ブレンド, 0.3986015319824219)","(塩梅, 0.3872663378715515)","(固め, 0.38112324476242065)","(工夫, 0.37132757902145386)"
料理 71,"(欲しい, 0.3825331926345825)","(cp, 0.36568310856819153)","(お通し, 0.36035120487213135)","(焼き物, 0.3591912090778351)","(緊張, 0.3542906641960144)","(時半, 0.35330748558044434)","(若い, 0.34453725814819336)","(かん, 0.34065279364585876)","(とく, 0.3381879925727844)","(感激, 0.33162617683410645)"
予約 71,"(回転, 0.6026515960693359)","(待ち, 0.5946770906448364)","(休日, 0.5373669862747192)","(到着, 0.5060124397277832)","(時半, 0.5059087872505188)","(当日, 0.4759616255760193)","(期間, 0.46611881256103516)","(来店, 0.44819584488868713)","(独立, 0.44388440251350403)","(ランチ, 0.4395105540752411)"
カウンター 68,"(個室, 0.5888069868087769)","(部屋, 0.5825007557868958)","(店内, 0.5576155185699463)","(デート, 0.5532745122909546)","(テーブル, 0.5287970900535583)","(清潔, 0.5102343559265137)","(移動, 0.49962612986564636)","(地下, 0.4977991580963135)","(入り口, 0.48843544721603394)","(アプローチ, 0.4768557846546173)"


In [43]:
def makedf_dinner(i):
    df = pd.DataFrame({many_words_dinner[i][0]:similar_words_dinner[i][0]})
    for t in range(1, len(similar_words_dinner[0])):
        df2 = pd.DataFrame({many_words_dinner[i][t]:similar_words_dinner[i][t]})
        df = pd.concat([df, df2], axis=1)
    return df.T

In [44]:
dinner = []
for i in range(len(store_name)): 
    dinner.append(makedf_dinner(i))

In [50]:
for i in range(len(dinner)):
    dinner[i].to_csv("pre_baba_frequency_"+store_name[i]+"_dinner.csv", index=True)

In [51]:
for i in range(len(lunch)):
    lunch[i].to_csv("pre_baba_frequency_"+store_name[i]+"_lunch.csv", index=True)
    

In [14]:
l.append("sushi"+" " + str(300))

In [15]:
l

['sushi300', 'sushi 300']

In [52]:
df = pd.read_csv("pre_baba_frequency_aozora_dinner.csv")

In [53]:
df

,Unnamed: 0,0,1,2,3,4,5,6,7,8,9
0,握り 138,"('構成', 0.46630582213401794)","('時半', 0.46482110023498535)","('焼き物', 0.4362568259239197)","('厳選', 0.4326210021972656)","('ドリンク', 0.4073576331138611)","('回転', 0.40107160806655884)","('かす', 0.3993162512779236)","('アジ', 0.39497530460357666)","('イクラ', 0.3903543949127197)","('身中', 0.3888811767101288)"
1,シャリ 129,"('存在', 0.5280452966690063)","('酸味', 0.5157119035720825)","('温度', 0.4781256914138794)","('個性', 0.4613175094127655)","('提示', 0.45696502923965454)","('強い', 0.4401971101760864)","('スッ', 0.4346611499786377)","('塩気', 0.4257599413394928)","('安定', 0.422967791557312)","('最上級', 0.4181242883205414)"
2,穴子 86,"('人生', 0.5262246131896973)","('いくら', 0.4876113533973694)","('馬糞', 0.46610039472579956)","('雲丹', 0.45786476135253906)","('過去', 0.45242512226104736)","(')○', 0.446027934551239)","('対馬', 0.4439021944999695)","('ムラサキ', 0.43822672963142395)","('うに', 0.4353557825088501)","('玉子', 0.4347919225692749)"
3,トロ 83,"('赤身', 0.6673526763916016)","('マグロ', 0.587617039680481)","('小柱', 0.5822740793228149)","('いくら', 0.5697892904281616)","('バフンウニ', 0.5235371589660645)","('身中', 0.5027121901512146)","('まぐろ', 0.492424339056015)","('舞鶴', 0.4861847460269928)","('管理', 0.47780609130859375)","('漬け', 0.4732981324195862)"
4,銀座 80,"('丁目', 0.5727304220199585)","('silver', 0.5655996799468994)","('代表', 0.5626869201660156)","('場所', 0.558918833732605)","('雑居', 0.5521233081817627)","('有名', 0.5517609119415283)","('記念', 0.545261025428772)","('コリドー', 0.5446872115135193)","('オープン', 0.5435529947280884)","('カップル', 0.5394082069396973)"
5,青空 69,"('オープン', 0.556469202041626)","('silver', 0.5428857207298279)","('ミシュラン', 0.5334504842758179)","('はる', 0.5190062522888184)","('たか', 0.5154526233673096)","('丁目', 0.4866343140602112)","('店内', 0.4801182746887207)","('シースー', 0.46821486949920654)","('コリドー', 0.4679040312767029)","('新しい', 0.458149254322052)"
6,大将 56,"('担当', 0.4837651252746582)","('ボラ', 0.4383164942264557)","('若い', 0.4336552619934082)","('弟子', 0.4299933910369873)","('カップル', 0.4224023222923279)","('ハードル', 0.4103327989578247)","('所作', 0.4044836759567261)","('秀逸', 0.3973230719566345)","('有名', 0.3962186872959137)","('辛口', 0.39135074615478516)"
7,ネタ 49,"('固め', 0.5188826322555542)","('評価', 0.43516063690185547)","('堪能', 0.42067450284957886)","('うまい', 0.41698721051216125)","('レベル', 0.4169676899909973)","('ブレンド', 0.40181583166122437)","('シャリ', 0.39886075258255005)","('おいしい', 0.3988511562347412)","('使い', 0.39405518770217896)","('接客', 0.38182902336120605)"
8,すき 38,"('やばし', 0.9546744227409363)","('次郎', 0.7770110368728638)","('修行', 0.6929305195808411)","('二郎', 0.6246300339698792)","('ミシュランガイド', 0.59489905834198)","('名店', 0.5904625654220581)","('オープン', 0.5655406713485718)","('獲得', 0.5630698800086975)","('出身', 0.562400221824646)","('たまらない', 0.5599291920661926)"
9,やばし 38,"('すき', 0.9546744227409363)","('次郎', 0.7675986886024475)","('修行', 0.6868759989738464)","('二郎', 0.6199650764465332)","('名店', 0.5864446759223938)","('ミシュランガイド', 0.5654603242874146)","('修業', 0.5558854341506958)","('獲得', 0.5353963375091553)","('オープン', 0.5343657732009888)","('出身', 0.5255336761474609)"
